# 第三章 提示优化工具

<div class="toc">
 <ul class="toc-item">
     <li><span><a href="#一、引言" data-toc-modified-id="一、引言">一、引言</a></span></li>
     <li>
         <span><a href="#二、优化 Prompt 的代码示例" data-toc-modified-id="二、优化 Prompt 的代码示例">二、优化 Prompt 的代码示例</a></span>
         <ul class="toc-item">
             <li><span><a href="##2.1 示例 1" data-toc-modified-id="2.1 示例 1">2.1 示例 1</a></span></li>
             <li><span><a href="##2.2 示例 2" data-toc-modified-id="2.2 示例 2">2.2 示例 2</a></span></li>
             <li><span><a href="##2.3 示例 3" data-toc-modified-id="2.3 示例 3">2.3 示例 3</a></span></li>
             </ul>
         </li>
     <li><span><a href="#三、总结与讨论" data-toc-modified-id="三、总结与讨论">三、总结与讨论</a></span></li>
     </ul>
</div>

## 一、引言
***

对于大语言模型（Large Language Models, LLMs），**我们通常需要遵循一些原则和策略来创建能产生高质量回答的有效提示：**

1. 明确和具体：你应该尽可能明确和具体地描述你希望 LLM 执行的任务。如果你期望得到特定类型的回答，应在提示中明确指出。同时，如果存在任何具体的限制或要求，也应明确提出。

2. 开放式与封闭式：你可以根据需求选择开放式问题（允许多种回答）或封闭式问题（限制回答范围）。这两种类型的问题各有优势，你应根据实际需求进行选择。

3. 语境清晰：为了让 LLM 生成更有意义的回答，你需要确保提供足够的上下文。如果提示是基于之前的信息，那么这些信息应被包含在内。

4. 创造力和想象力：如果你期望得到创新性的输出，可以鼓励人工智能进行发散思维或头脑风暴。如果这符合你的需求，你还可以建议人工智能模拟某些场景。

**实际上，我们可以利用这些原则和策略来有效地优化提示。**

本章将介绍一个适用于 LLM 的优化提示工具。这个工具能够帮助你像 [PromptPerfect](https://promptperfect.jina.ai/home) 那样有效地优化提示，从而最大限度地发挥 LLM 的潜力，并获得更准确、与上下文相关的回答。


## 二、优化 Prompt 的代码示例
***
注意：我使用的测试环境是 UV + Python3.12+。Python 的版本要求 Python3.12+。

开始实验之前，我们需要按照`Env-Setup.md`中，安装好依赖环境。OpenAI Python 库提供了一个便捷的途径，让任何 Python 3.12+ 应用程序可以访问 OpenAI REST API。

导入此次实验所需的依赖库

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
from IPython.display import Markdown

loaded = load_dotenv(find_dotenv(), override=True)
# 从环境变量中获取 OpenAI API Key 或者直接赋值
API_KEY = os.getenv("API_KEY")

# 如果您使用的是官方 API，就直接用 https://api.siliconflow.cn/v1 就行。
BASE_URL = "https://api.siliconflow.cn/v1"
# 如果您使用的不是官方 API，而是通过代理进行请求，请设置您的代理 URL。
# BASE_URL = "https://api.xxx.../v1"

In [2]:
# 实例化 OpenAI 对象
# 传入参数：OpenAI API Key（必需）、Base URL 和最大重试次数
client = OpenAI(api_key=API_KEY, base_url=BASE_URL, max_retries=3)

In [3]:
# 参数 n，整数或 Null，可选项，默认为 1。为每条输入信息生成多少个聊天完成选项。
# 参数 temperature，实数值或 Null，可选项，默认为 1。使用的采样温度，介于 0 和 2 之间。0.8 等较高值会使输出更加随机，而 0.2 等较低值会使输出更加集中和确定。

def get_completions1(system_instruction, llm_prompt, model_endpoint):

    extra_body = {}
    if "Qwen3" in model_endpoint:
        extra_body={
            "enable_thinking": False
        }
    
    response = client.chat.completions.create(model=model_endpoint,
                                              messages=[{"role": "system", "content": system_instruction},
                                                        {"role": "user", "content": llm_prompt}
                                                       ],
                                              n=1, temperature=0.60, seed=42,
                                              presence_penalty=0, frequency_penalty=0,
                                              max_tokens=1024, extra_body = extra_body
                                             )

    return response.choices[0].message.content.strip()


def get_completions2(system_instruction, llm_prompt, model_endpoint):

    extra_body = {}
    if "Qwen3" in model_endpoint:
        extra_body={
            "enable_thinking": False
        }
    
    response = client.chat.completions.create(model=model_endpoint,
                                              messages=[{"role": "system", "content": system_instruction},
                                                        {"role": "user", "content": llm_prompt}
                                                       ],
                                              n=1, temperature=0, seed=42,
                                              presence_penalty=0, frequency_penalty=0,
                                              max_tokens=1024, extra_body = extra_body
                                             )

    return response.choices[0].message.content.strip()

下面给出的 `system_instruction` 模板的本质是利用**上下文学习**（Few-shot 演示）和提示工程。

In [4]:
# 中文
system_instruction1 = f"""你是一位出色的人工智能提示工程师。你是设计 ChatGPT 提示的专家，能取得最佳效果。

要创建能产生高质量回答的有效提示，请考虑以下原则和策略：
1. 明确和具体：尽可能明确、具体地说明你希望人工智能做些什么。如果你想要某种类型的答复，请在提示中简要说明。如果有具体的限制或要求，也一定要写明。
2. 开放式与封闭式：根据你的需求，你可能会选择开放式问题（允许多种回答）或封闭式问题（缩小可能回答的范围）。两者都有其用途，请根据自己的需要进行选择。
3. 语境清晰：确保提供足够的上下文，以便人工智能生成有意义的相关回复。如果提示是基于先前的信息，则应确保包含这些信息。
4. 创造力和想象力：如果你想要创造性的输出，可以鼓励人工智能发散思维或进行头脑风暴。如果符合你的需求，你甚至可以建议人工智能想象某些场景。

你的任务是根据用户给定的提示进行优化，设计出一个新的、经过优化的 ChatGPT 提示。

下面提供了一些好的示例，供你参考：
原始提示：你的任务是使用 Python，根据用户给的数据绘制散点图。
优化后的提示：你的任务是使用 Python 编程语言，根据用户给定的数据绘制一个散点图。请确保散点图能够清晰地展示数据之间的关系，并使用适当的颜色、形状和标签来区分不同的数据点。此外，你还需要为图表添加标题、坐标轴标签以及图例（如果适用），以便更好地解释数据的含义。最后，请将生成的散点图保存为一个图像文件，以便用户可以查看和分享。
原始提示：你的任务是将用户给定的英文翻译成中文。
优化后的提示：请将以下英文文本准确地翻译成中文，保持原文的意思和语境不变。如果英文中包含专有名词或术语，请在翻译时注明其标准中文名称或提供相应的解释。此外，请确保翻译后的中文句子通顺、语法正确，并且符合中文的表达习惯。
原始提示：你的任务是指导初学者系统地学习机器学习。
优化后的提示：你被要求为一个完全没有机器学习背景的初学者设计一个详细的学习计划。请你描述这个计划，包括以下维度：首先，推荐适合初学者的机器学习入门教材或在线课程；其次，提供一个结构化的学习路线图，明确指出在学习过程中应该先掌握哪些基本概念，再逐步深入到哪些高级主题；最后，请提出一些实践建议，比如初学者可以通过哪些项目来巩固理论知识，以及在遇到困难时如何寻求帮助和资源。请确保你的回答涵盖了机器学习的基础知识、算法理解、编程实践，以及如何解决实际问题等方面。
"""

# 英文
system_instruction2 = f"""You are an exceptional AI prompt engineer. \
You are an expert at writing ChatGPT Prompts for best results.

To create efficient prompts that yield high-quality responses, \
consider the following principles and strategies:
1. Clear and Specific: Be as clear and specific as possible about what you want from the AI. If you want a certain type of response, outline that in your prompt. If there are specific constraints or requirements, make sure to include those as well.
2. Open-ended vs. Closed-ended: Depending on what you're seeking, you might choose to ask an open-ended question (which allows for a wide range of responses) or a closed-ended question (which narrows down the possible responses). Both have their uses, so choose according to your needs.
3. Contextual Clarity: Make sure to provide enough context so that the AI can generate a meaningful and relevant response. If the prompt is based on prior information, ensure that this is included.
4. Creativity and Imagination: If you want creative output, encourage the AI to think outside the box or to brainstorm. You can even suggest the AI to imagine certain scenarios if it fits your needs.

Your task is to design a new, optimized ChatGPT prompt based on the user-given prompt.
Some good examples are provided below for your reference:
Original prompt: Your task is to plot a scatterplot based on data given by the user, using Python.
Optimized prompt: Your task is to use the Python programming language to draw a scatterplot based on data given by a user. Make sure that the scatterplot clearly shows the relationships between the data and uses appropriate colors, shapes, and labels to distinguish the different data points. In addition, you will need to add a title, axis labels, and a legend (if applicable) to the chart to better explain the meaning of the data. Finally, please save the resulting scatterplot as an image file so that users can view and share it.
Original prompt: Your task is to translate the English text given by the user into Chinese.
Optimized prompt: Please translate the following English text accurately into Chinese, keeping the meaning and context of the original text intact. If the English text contains proper nouns or terms, please indicate their standard Chinese names or provide corresponding explanations when translating. In addition, please make sure that the translated Chinese sentences are fluent, grammatically correct, and in line with Chinese expressions.
Original prompt: You have been tasked with guiding a beginner to learn machine learning in a systematic way.
Optimized prompt: You have been asked to design a detailed learning plan for a beginner who has no machine learning background at all. Please describe the plan, including the following dimensions: first, recommend introductory machine learning textbooks or online courses suitable for beginners; second, provide a structured learning roadmap that clearly indicates which basic concepts should be mastered before progressively delving into which advanced topics during the learning process; finally, please make some practical suggestions, such as which projects beginners can work on to consolidate their theoretical knowledge and how to seek help and resources when they are how to seek help and resources when encountering difficulties. Please make sure your answer covers the basics of machine learning, algorithmic understanding, programming practices, and how to solve real-world problems.
"""

### 2.1 示例 1

In [5]:
llm = "Qwen/Qwen3-8B"

original_prompt = "你的任务是使用 Python，根据用户给的数据绘制热力图。"
original_prompt = f"""原始提示：{original_prompt}"""

optimized_prompt = get_completions1(system_instruction1, original_prompt, llm)
print(f"{original_prompt}\n")
print(f"{optimized_prompt}\n")
print("-" * 100)
instruction_ = "你是世界知识专家。"
result1 = get_completions2(instruction_, original_prompt, llm)
result2 = get_completions2(instruction_, optimized_prompt, llm)

print(f"原始提示的生成结果：\n{result1}\n")
print("-" * 100)
print(f"优化后的提示的生成结果：\n{result2}")

原始提示：你的任务是使用 Python，根据用户给的数据绘制热力图。

优化后的提示：你的任务是使用 Python 编程语言，根据用户提供的数据集绘制一个清晰且具有视觉表现力的热力图。请确保热力图能够有效展示数据中的相关性或密度分布，并使用合适的颜色映射（如从浅到深的渐变色）来增强可读性。此外，你还需要为图表添加标题、坐标轴标签以及图例（如果适用），以便更好地解释数据的含义。如果数据维度较多，请考虑使用适当的注释或交互方式来帮助用户理解。最后，请将生成的热力图保存为图像文件，并提供一个简要的说明，解释热力图中颜色和模式所代表的含义。

----------------------------------------------------------------------------------------------------
原始提示的生成结果：
当然可以！使用 Python 绘制热力图（Heatmap）是一个非常常见的任务，通常可以通过 `matplotlib` 和 `seaborn` 库来实现。下面我将为你提供一个完整的示例，包括数据准备、热力图绘制以及一些基本的样式设置。

---

## ✅ 一、热力图简介

热力图是一种用颜色表示数据矩阵中数值大小的图表，常用于可视化二维数据，比如相关系数矩阵、矩阵数据分布等。颜色越深，表示数值越大；颜色越浅，表示数值越小。

---

## ✅ 二、Python 绘制热力图的常用库

- **matplotlib**：基础绘图库，可以绘制热力图。
- **seaborn**：基于 matplotlib 的高级库，提供更简洁的 API 来绘制热力图。
- **pandas**：用于数据处理和整理，特别是处理表格数据。

---

## ✅ 三、示例：使用 Seaborn 绘制热力图

### 📌 1. 安装必要的库（如果尚未安装）

```bash
pip install matplotlib seaborn pandas numpy
```

### 📌 2. 示例数据准备

我们可以使用 `numpy` 生成一个随机的 10x10 矩阵作为示例数据：

```python
import numpy as np
import seaborn as sns
import matplotlib.pyplot as p

In [6]:
llm = "Qwen/Qwen3-8B"

# 英文
original_prompt = "Your task is to use Python and draw a heat map based on the data given by the user."
original_prompt = f"""Original prompt: {original_prompt}"""

optimized_prompt = get_completions1(system_instruction2, original_prompt, llm)
print(f"{original_prompt}\n")
print(f"{optimized_prompt}\n")
print("-" * 100)
instruction_ = "You are an expert at world knowledge."
result1 = get_completions2(instruction_, original_prompt, llm)
result2 = get_completions2(instruction_, optimized_prompt, llm)

print(f"原始提示的生成结果：\n{result1}\n")
print("-" * 100)
print(f"优化后的提示的生成结果：\n{result2}")

Original prompt: Your task is to use Python and draw a heat map based on the data given by the user.

Optimized prompt: Your task is to use Python to generate a heat map based on the data provided by the user. Please ensure that the heat map is visually clear and effectively represents the data's distribution or correlation. Use appropriate color schemes to highlight patterns, add axis labels, a title, and a legend (if applicable) to enhance interpretability. If the data includes multiple categories or variables, please structure the heat map accordingly. In addition, make sure to include any necessary data preprocessing steps and explain how the heat map helps in understanding the data. Finally, save the resulting heat map as an image file and provide the code used to generate it.

----------------------------------------------------------------------------------------------------
原始提示的生成结果：
Sure! I can help you create a heatmap using Python. To do that, I'll need the data you want to

### 2.2 示例 2

In [7]:
llm = "Qwen/Qwen3-8B"

original_prompt = "你的任务是将用户给定的中文翻译成英文。"
original_prompt = f"""原始提示：{original_prompt}"""

optimized_prompt = get_completions1(system_instruction1, original_prompt, llm)

instruction_ = "你是世界知识专家。"
content = """《蝴蝶书》力图帮助大家降低门槛，缩小应用程序和研究之间的差距，使得大模型应用开发变得触手可及。\
它是一本面向有一定编程基础或实际项目（不一定是算法）经历，\
对 ChatGPT（或类似模型）感兴趣、想利用相关技术做一些新产品或应用的学习者的书籍，\
旨在利用 ChatGPT API 开发相关应用。期望把方法传播给更多人，期望新技术的突破能够更多地改善我们所处的世界。"""
original_prompt = original_prompt + "\n" + f"<text>{content}</text>"
optimized_prompt = optimized_prompt + "\n" + f"<text>{content}</text>"
print(f"{original_prompt}\n")
print(f"{optimized_prompt}\n")
print("-" * 100)
result1 = get_completions2(instruction_, original_prompt, llm)
result2 = get_completions2(instruction_, optimized_prompt, llm)

print(f"原始提示的生成结果：\n{result1}\n")
print("-" * 100)
print(f"优化后的提示的生成结果：\n{result2}")

原始提示：你的任务是将用户给定的中文翻译成英文。
<text>《蝴蝶书》力图帮助大家降低门槛，缩小应用程序和研究之间的差距，使得大模型应用开发变得触手可及。它是一本面向有一定编程基础或实际项目（不一定是算法）经历，对 ChatGPT（或类似模型）感兴趣、想利用相关技术做一些新产品或应用的学习者的书籍，旨在利用 ChatGPT API 开发相关应用。期望把方法传播给更多人，期望新技术的突破能够更多地改善我们所处的世界。</text>

优化后的提示：  
请将以下中文文本准确、自然地翻译成英文，保持原意和语境不变。如果原文中包含专有名词、技术术语或文化特定表达，请确保使用合适的英文对应词汇或进行适当解释。此外，请注意英文句子的语法结构和表达习惯，使译文通顺、专业，并适合用于正式或学术场合。如果需要，你可以使用工具辅助翻译，但请优先保证翻译的准确性和地道性。最后，请将翻译结果以清晰的格式呈现，并在必要时提供中英文对照以便用户理解。
<text>《蝴蝶书》力图帮助大家降低门槛，缩小应用程序和研究之间的差距，使得大模型应用开发变得触手可及。它是一本面向有一定编程基础或实际项目（不一定是算法）经历，对 ChatGPT（或类似模型）感兴趣、想利用相关技术做一些新产品或应用的学习者的书籍，旨在利用 ChatGPT API 开发相关应用。期望把方法传播给更多人，期望新技术的突破能够更多地改善我们所处的世界。</text>

----------------------------------------------------------------------------------------------------
原始提示的生成结果：
The *Butterfly Book* aims to help readers lower the barrier and narrow the gap between applications and research, making large model application development more accessible. It is a book for learners who have some programming foundation or practical project experience (not 

In [8]:
llm = "Qwen/Qwen3-8B"

original_prompt = "Your task is to translate the Chinese given by the user into English."
original_prompt = f"""Original prompt: {original_prompt}"""

optimized_prompt = get_completions2(system_instruction2, original_prompt, llm)

instruction_ = "You are an expert at world knowledge."
content = """《蝴蝶书》力图帮助大家降低门槛，缩小应用程序和研究之间的差距，使得大模型应用开发变得触手可及。\
它是一本面向有一定编程基础或实际项目（不一定是算法）经历，\
对 ChatGPT（或类似模型）感兴趣、想利用相关技术做一些新产品或应用的学习者的书籍，\
旨在利用 ChatGPT API 开发相关应用。期望把方法传播给更多人，期望新技术的突破能够更多地改善我们所处的世界。"""
original_prompt = original_prompt + "\n" + f"<text>{content}</text>"
optimized_prompt = optimized_prompt + "\n" + f"<text>{content}</text>"
print(f"{original_prompt}\n")
print(f"{optimized_prompt}\n")
print("-" * 100)
result1 = get_completions2(instruction_, original_prompt, llm)
result2 = get_completions2(instruction_, optimized_prompt, llm)

print(f"原始提示的生成结果：\n{result1}\n")
print("-" * 100)
print(f"优化后的提示的生成结果：\n{result2}")

Original prompt: Your task is to translate the Chinese given by the user into English.
<text>《蝴蝶书》力图帮助大家降低门槛，缩小应用程序和研究之间的差距，使得大模型应用开发变得触手可及。它是一本面向有一定编程基础或实际项目（不一定是算法）经历，对 ChatGPT（或类似模型）感兴趣、想利用相关技术做一些新产品或应用的学习者的书籍，旨在利用 ChatGPT API 开发相关应用。期望把方法传播给更多人，期望新技术的突破能够更多地改善我们所处的世界。</text>

Optimized prompt: Please translate the following Chinese text accurately into English, ensuring that the meaning, tone, and context of the original are preserved. If the text contains idiomatic expressions, cultural references, or specific terminology, please provide appropriate translations that are natural and commonly understood in English. In addition, make sure that the translated sentences are grammatically correct, fluent, and maintain the original intent. If there are any ambiguous phrases, please clarify them in your translation and explain the reasoning behind your choices.
<text>《蝴蝶书》力图帮助大家降低门槛，缩小应用程序和研究之间的差距，使得大模型应用开发变得触手可及。它是一本面向有一定编程基础或实际项目（不一定是算法）经历，对 ChatGPT（或类似模型）感兴趣、想利用相关技术做一些新产品或应用的学习者的书籍，旨在

### 2.3 示例 3

In [9]:
llm = "Qwen/Qwen3-8B"

original_prompt = "你的任务是指导初学者系统地学习数据科学。"
original_prompt = f"""原始提示：{original_prompt}"""

optimized_prompt = get_completions1(system_instruction1, original_prompt, llm)
print(f"{original_prompt}\n")
print(f"{optimized_prompt}\n")
print("-" * 100)
instruction_ = "你是世界知识专家。"
result1 = get_completions2(instruction_, original_prompt, llm)
result2 = get_completions2(instruction_, optimized_prompt, llm)

print(f"原始提示的生成结果：\n{result1}\n")
print("-" * 100)
print(f"优化后的提示的生成结果：\n{result2}")

原始提示：你的任务是指导初学者系统地学习数据科学。

优化后的提示：你被要求为一个完全没有数据科学背景的初学者设计一个全面且结构化的学习路径。请描述一个循序渐进的学习计划，涵盖以下方面：首先，推荐适合初学者的数据科学入门教材、在线课程或资源；其次，明确列出学习过程中需要掌握的核心概念和技能，包括编程基础（如 Python）、数学知识（如统计学、线性代数）、数据分析工具（如 Pandas、NumPy）、数据可视化（如 Matplotlib、Seaborn）以及机器学习基础（如监督与非监督学习）；最后，请提供一些实践建议，比如初学者可以通过哪些真实项目来应用所学知识，以及在学习过程中遇到困难时应如何有效寻求帮助和资源。请确保你的回答逻辑清晰、内容详实，并能帮助初学者建立扎实的数据科学基础，逐步提升其分析和解决问题的能力。

----------------------------------------------------------------------------------------------------
原始提示的生成结果：
当然！作为世界知识专家，我可以为你提供一个系统化的学习路径，帮助初学者逐步掌握数据科学的核心概念和技能。以下是为初学者设计的数据科学学习路线，分为几个阶段，涵盖理论、工具、实践和项目。

---

## 🌟 一、数据科学概述（入门阶段）

### 1.1 什么是数据科学？
- 数据科学是利用数据来发现知识、做出决策和预测未来的学科。
- 涉及领域包括统计学、机器学习、数据可视化、数据库管理、编程等。

### 1.2 数据科学的应用场景
- 金融：风险评估、欺诈检测
- 医疗：疾病预测、药物研发
- 电商：用户行为分析、推荐系统
- 社交媒体：内容推荐、用户画像
- 交通：路线优化、交通预测

### 1.3 学习资源推荐
- 书籍：《Python for Data Analysis》（Wes McKinney）、《Storytelling with Data》（Cole Nussbaumer Knack）
- 在线课程：Coursera、edX、Udemy、Kaggle Learn
- 网站：Kaggle、Towards Data Science、DataCamp、Medium

---

## 🧠 二、数学基础（核心阶段）



In [10]:
llm = "Qwen/Qwen3-8B"

original_prompt = "Your task is to guide beginners through the systematic study of data science."
original_prompt = f"""Original prompt: {original_prompt}"""

optimized_prompt = get_completions1(system_instruction2, original_prompt, llm)
print(f"{original_prompt}\n")
print(f"{optimized_prompt}\n")
print("-" * 100)
instruction_ = "You are an expert at world knowledge."
result1 = get_completions2(instruction_, original_prompt, llm)
result2 = get_completions2(instruction_, optimized_prompt, llm)

print(f"原始提示的生成结果：\n{result1}\n")
print("-" * 100)
print(f"优化后的提示的生成结果：\n{result2}")

Original prompt: Your task is to guide beginners through the systematic study of data science.

Optimized prompt: You have been asked to design a comprehensive and structured learning plan for beginners who are completely new to data science. Please outline the plan, including the following components: first, recommend beginner-friendly data science resources such as books, online courses, and tutorials that cover the foundational concepts; second, provide a step-by-step learning roadmap that progresses from basic skills like data manipulation and visualization to more advanced topics such as machine learning and statistical analysis; finally, suggest practical projects and exercises that beginners can undertake to apply their knowledge and gain hands-on experience. Make sure your plan is easy to follow, covers essential tools like Python, R, or SQL, and includes tips on how to seek help and stay motivated while learning. Also, emphasize the importance of problem-solving and real-world

## 三、总结与讨论
***

**优化 ChatGPT 的 Prompts 对于提升交互体验和生成结果的质量非常重要。**以下是一些主要的原因：
- 提高准确性：通过优化 Prompts，我们可以更准确地引导 ChatGPT 生成我们期望的回答。这对于获取准确、相关和有用的信息至关重要。

- 提高效率：优化的 Prompts 可以减少无效或者不相关的回答，从而提高交互的效率。

- 增强可控性：通过优化 Prompts，我们可以更好地控制 ChatGPT 的输出，使其更符合用户的需求和期望。

- 提升用户体验：优化的 Prompts 可以帮助 ChatGPT 更好地理解用户的需求，从而提供更满意的回答，提升用户体验。

总的来说，优化 Prompts 是提高 ChatGPT 性能和用户满意度的关键步骤。它有助于我们更好地利用提示工程（Prompt Engineering），为用户提供高质量的服务。

本章将介绍一个适用于 LLM 的优化提示工具。这个工具能够帮助你像 [PromptPerfect](https://promptperfect.jina.ai/home) 那样有效地优化提示，从而最大限度地发挥 LLM 的潜力，并获得更准确、与上下文相关的回答。

在实际应用中，为 LLM 提供的演示对于优化 Prompt 的性能具有重要意义。对于特定需求，如果手动构建优质的演示（**成对的原始提示和优化后的提示**）存在困难，我们可以利用一些商业化（非开源）的提示优化工具，如：[PromptPerfect](https://promptperfect.jina.ai/home)、[百度千帆大模型操作台的 Prompt 优化](https://cloud.baidu.com/doc/WENXINWORKSHOP/s/Clommng91)、[讯飞星火的指令优化](https://xinghuo.xfyun.cn/desk)等。这些商业工具通常进行了大量的工程优化，能够适应各种场景，满足用户的特定需求。
